In [4]:
import os
import os.path as osp
import networkx as nx
import json
import matplotlib.pyplot as plt
import hashlib
import shutil
import subprocess


from utils.map_utils import get_game_info
from utils.eval_gpt_utils import *
from utils.clean_utils import *

In [5]:
# notebook input
map_dir='/share/data/mei-work/kangrui/github/mango/data/'
result_dir='/share/data/mei-work/kangrui/github/mango/kangrui/data/gpt-games-results-clean-new-new'

In [2]:
# specific functions for data cleaning
def get_collection_and_drop_rf(game_name,map_dir,final_dir_path):
    assert game_name in final_dir_path
    G,actions,locations,all2all,all_pairs,walkthrough=get_game_info(map_dir,game_name)
    
    pair_id_set=get_pair_id_set(all_pairs)

   
    drop_list=[]
    collection={}
    for idx,file_name in enumerate(os.listdir(final_dir_path)):

        final_file_path=osp.join(final_dir_path,file_name)
        try:
            pair_dict=extract_pair_from_file(final_file_path)
        except Exception as e:
            print(final_file_path,e)
        pair_id=compute_hash_for_pair(pair_dict)

        if pair_id in pair_id_set:
            if pair_id in collection.keys():
                collection[pair_id].append(final_file_path)
            else:
                collection[pair_id]=[final_file_path]
        else:
            drop_list.append(final_file_path)
    
    collection_dup={}
    for k,v in collection.items():
        if len(v)>1:
            collection_dup[k]=v
    return drop_list,collection_dup

def handle_dup_drop_rf(game_name,map_dir,collection_dup):
    G,actions,locations,all2all,all_pairs,walkthrough=get_game_info(map_dir,game_name)
#     print([len(v) for k,v in collection_dup.items()])
    for k,v in collection_dup.items():
        flag=True
        for idx,file in enumerate(v):
            
            try:
                flag_easy,flag_harsh,flag_hard=eval_rf(file,G,all2all,all_pairs,eval_difficulty='loose')
            except Exception as e:
                print(file,e)
                continue
            
            if flag_easy==1 or flag_harsh==1:
                collection_dup[k].pop(idx)
                flag=False
                print("right answer found")
                break
        if flag:
            collection_dup[k].pop()
#     print([len(v) for k,v in collection_dup.items()])
    for k,v in collection_dup.items():
        for file_name in v:
            os.remove(file_name)

In [4]:
import os
for idx,game_name in enumerate(os.listdir(result_dir)):
    print(f'checking {game_name} ...')
    G,actions,locations,all2all,all_pairs,walkthrough=get_game_info(map_dir,game_name)
    
    # get path id set in a game
    pair_id_set=get_pair_id_set(all_pairs)
    
    # iterate pathgen in result-clean
    file_dir=osp.join(result_dir,game_name,'results')
    
#     print(file_dir)
    # iterate pathgen for gpt3 and gpt4
    for idx,dir_path in enumerate(os.listdir(file_dir)):
        if not dir_path.startswith("pathgen"):
            continue
            
        final_dir_path=osp.join(file_dir,dir_path)
        drop_list,collection_dup=get_collection_and_drop_rf(game_name,map_dir,final_dir_path)
        if len(collection_dup.keys())>0:
            handle_dup_drop_rf(game_name,map_dir,collection_dup)
#         os.sys.exit()
        for file_name in drop_list:
            os.remove(file_name)

checking awaken ...
checking planetfall ...
checking zenon ...
checking yomomma ...
checking omniquest ...
checking hhgg ...
checking huntdark ...
checking cutthroat ...
checking ztuu ...
checking advent ...
checking lostpig ...
checking deephome ...
checking partyfoul ...
checking snacktime ...
checking inhumane ...
checking 905 ...
checking pentari ...
checking balances ...
checking spirit ...
checking loose ...
checking hollywood ...
checking library ...
checking afflicted ...
checking enchanter ...
checking jewel ...
checking infidel ...
checking zork3 ...
checking lurking ...
checking ludicorp ...
checking reverb ...
checking seastalker ...
checking tryst205 ...
checking detective ...
checking zork1 ...
checking anchor ...
checking karn ...
checking plundered ...
checking temple ...
checking spellbrkr ...
checking adventureland ...
checking wishbringer ...
checking enter ...
checking gold ...
checking trinity ...
checking murdac ...
checking ballyhoo ...
checking zork2 ...
checkin